## Snapshot2D + BIDS

This is an example that creates snapshots for a dataset with MRI (dixon) and CT.

In [ ]:
from TPTBox.snapshot2D import create_snapshot, Snapshot_Frame, Visualization_Type, Image_Modes
from TPTBox import BIDS_Global_info, calc_centroids_labeled_buffered, Location
from pathlib import Path
from IPython.display import Image

# Warning: This file loops over a whole dataset and makes snapshots
dataset = Path("/media/data/robert/datasets/spine_transition_new/")

# Running the docker
You must have installed the docker

In [ ]:
from TPTBox import run_docker

run_docker(dataset, verbose=True)

# Loop over all files and make a snapshot (CT/MRI)

In [ ]:
bids_global_object = BIDS_Global_info(
    [dataset], ["rawdata", "derivatives"], additional_key=["sequ", "seg", "ovl", "e"], verbose=True, clear=True
)
# First loop: Loop over subjects
for subject_name, subject_container in bids_global_object.enumerate_subjects(sort=True):
    ct_query = subject_container.new_query(flatten=True)
    ct_query.filter("format", "ct")
    ct_query.filter("Filetype", "nii.gz")
    dixon_query = subject_container.new_query(flatten=True)
    dixon_query.filter("format", "dixon")
    # dixon_query.flatten()
    dixon_query.filter("Filetype", "nii.gz")
    dixon_query.filter("Filetype", "json")
    # .filter("json", lamda) opens the json and gives it as an argument.
    # "IP" is in the list with key "ImageType" in the json if and only if it is the inphase image
    dixon_query.filter("json", filter_fun=lambda x: "IP" in x["ImageType"])  # type: ignore

    # now we can loop over the CT files.
    for ct in ct_query.loop_list():
        for dixon in dixon_query.loop_list():
            ct_family = ct.get_sequence_files()
            vert = ct_family.get("msk_vert", None)
            subreg = ct_family.get("msk_subreg", None)
            vert = vert[0] if vert is not None else None
            subreg = subreg[0] if subreg is not None else None
            ctd = None
            if vert is not None and subreg is not None:
                ctd = calc_centroids_labeled_buffered(vert, subreg, subreg_id=Location.Vertebra_Corpus)

            frames = [
                Snapshot_Frame(dixon, mode="MRI"),
                Snapshot_Frame(ct, segmentation=vert, centroids=ctd, mode="CT"),
            ]  # without centroids
            out_file1 = ct.get_changed_path("jpg", format="snapshot", parent="derivatives")
            out_file2 = ct.dataset / "snapshots" / out_file1.name
            out_file2.parent.mkdir(exist_ok=True)
            create_snapshot([out_file1, out_file2], frames)
            # ct_frame = Snapshot_Frame(ct_file,segmentation=vert_file,centroids=ctd_file,sagittal=True,coronal=True,mode='CT')

In [ ]:
dataset = Path("/media/data/new_NAKO/NAKO/MRT/")

bids_global_object = BIDS_Global_info(
    [dataset], ["rawdata_low_quality_ds"], additional_key=["sequ", "seg", "ovl", "e"], verbose=True, clear=True
)
# First loop: Loop over subjects
files = []
for subject_name, subject_container in bids_global_object.enumerate_subjects(sort=True):
    t2_query = subject_container.new_query(flatten=True)
    t2_query.filter("format", "T2w")
    # dixon_query.flatten()
    t2_query.filter("Filetype", "nii.gz")
    # now we can loop over the CT files.
    regions = {}
    for t2w in t2_query.loop_list():
        c = t2w.get("chunk")
        out_file1 = t2w.get_changed_path("jpg", format="snapshot", parent="derivatives", make_parent=False)

        if c in regions:
            frames = [Snapshot_Frame(t2w, mode="MRI"), Snapshot_Frame(regions[c], mode="MRI")]  # without centroids
            out_file2 = t2w.dataset / "rawdata_low_quality_ds" / "snapshots_pairs" / out_file1.name
            out_file2.parent.mkdir(exist_ok=True)
            create_snapshot([out_file2], frames)
        regions[c] = t2w
        frames = [Snapshot_Frame(t2w, mode="MRI")]  # without centroids
        out_file2 = t2w.dataset / "rawdata_low_quality_ds" / "snapshots" / out_file1.name
        out_file2.parent.mkdir(exist_ok=True)
        create_snapshot([out_file2], frames)


print(files)

In [ ]:
for l in files:
    print(l)